In [1]:
import pandas as pd

from numpy import log, pi, sin
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
Tbg = 2.8
Twarm = 298
Thot = 346
Tground = 291

In [4]:
data = pd.read_csv('RRY100_2020_LAB1_elev_power.txt', names=['epsilon', 'Vant', 'Vwarm', 'Vhot'])

In [5]:
data['epsilon_rad'] = data.epsilon * pi / 180

In [6]:
data['V1'] = data.Vwarm - data.Vant
data['V2'] = data.Vwarm - data.Vhot
data['V1_over_V2'] = data.V1 / data.V2
DThot = 0
data['Tant'] = Twarm - data.V1_over_V2 * (Twarm - (Thot + DThot))

In [7]:
data

,epsilon,Vant,Vwarm,Vhot,epsilon_rad,V1,V2,V1_over_V2,Tant
0,90.0,2.010751,2.447681,2.511143,1.570796,0.436930,-0.063461,-6.884993,-32.479651
1,85.0,2.009611,2.448699,2.510240,1.483530,0.439088,-0.061542,-7.134829,-44.471808
2,80.0,2.010059,2.448046,2.511598,1.396263,0.437987,-0.063551,-6.891861,-32.809317
3,75.0,2.009558,2.446263,2.511631,1.308997,0.436705,-0.065368,-6.680709,-22.674042
4,70.0,2.012279,2.448377,2.509254,1.221730,0.436098,-0.060877,-7.163560,-45.850900
5,65.0,2.022005,2.447956,2.510601,1.134464,0.425951,-0.062645,-6.799430,-28.372661
6,60.0,2.019001,2.447020,2.511884,1.047198,0.428019,-0.064864,-6.598699,-18.737529
7,57.5,2.021100,2.448152,2.510803,1.003564,0.427052,-0.062651,-6.816412,-29.187775
8,55.0,2.022050,2.447101,2.508814,0.959931,0.425051,-0.061713,-6.887571,-32.603406
9,52.5,2.024213,2.449590,2.508553,0.916298,0.425377,-0.058962,-7.214393,-48.290860


## TODO

In [8]:
#data['M'] = 1 / sin(data.epsilon_rad)
#data['M_tauZ'] = - log((Teff - Tant) / (Teff - Tbg))

In [9]:
#p = figure(title='line', plot_width=1200, plot_height=600)

#p.line(data.M_tauZ, data.M, line_width=2)

#show(p)

In [10]:
#data['Y'] = Vhot / Vwarm
#data['Trec'] = (data.Thot - Y * data.Twarm) / (data.Y - 1)